In [1]:
!pip install pybaseball

  Using cached pybaseball-2.2.7-py3-none-any.whl.metadata (11 kB)
Using cached pybaseball-2.2.7-py3-none-any.whl (426 kB)


In [2]:
# Find Clayton Kershaw's player id
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
print(playerid_lookup('kershaw', 'clayton'))

# His MLBAM ID is 477132, so we feed that as the player_id argument to the following function 
kershaw_stats = statcast_pitcher('2017-06-01', '2017-07-01', 477132)
print(kershaw_stats.groupby("pitch_type").release_speed.agg("mean"))




# from pybaseball import statcast
# statcast(start_dt="2019-06-24", end_dt="2019-06-25").columns
# Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
#        'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
#        'description', 'spin_dir', 'spin_rate_deprecated',
#        'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
#        'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
#        'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
#        'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
#        'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
#        'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
#        'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
#        'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
#        'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
#        'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
#        'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
#        'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
#        'woba_value', 'woba_denom', 'babip_value', 'iso_value',
#        'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name',
#        'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
#        'post_home_score', 'post_bat_score', 'post_fld_score',
#        'if_fielding_alignment', 'of_fielding_alignment', 'spin_axis',
#        'delta_home_win_exp', 'delta_run_exp'],
#       dtype='object')

Gathering player lookup table. This may take a moment.
  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0   kershaw    clayton     477132  kersc001  kershcl01           2036   

   mlb_played_first  mlb_played_last  
0            2008.0           2025.0  
Gathering Player Data
pitch_type
CH    86.725000
CU    73.133333
FF    92.844622
SI    94.515385
SL    87.962381
Name: release_speed, dtype: float64
